In [ ]:
%%R
library(reticulate)
library(knitr)
opts_chunk$set(message = TRUE, echo = FALSE, warning = TRUE, error = TRUE)

In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None 
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import os
import datetime
#os.environ['QT_QPA_PLATFORM_PLUGIN_PATH'] = 'C:/Users/user/Anaconda3/Library/plugins/platforms'
np.random.seed(seed=1994)

In [ ]:
#Download data with PDR
end = datetime.datetime.today()
start = datetime.datetime(year=end.year-10, month=end.month, day=end.day)

#Fred as standard API
symbols = ["T5YIE","T10YIE"]
fred = pdr.DataReader(symbols, 'fred', start, end)
symbols = ["PCEPI"]
pcepi_index = pdr.DataReader(symbols, 'fred', start, end)
pcepi_index.index = pcepi_index.index + pd.offsets.MonthEnd(0) 
fred = pd.concat([fred, pcepi_index], axis = 1)
fred.to_csv("fred.csv")

fred = pd.read_csv("fred.csv", index_col = "DATE")

#Cleanse Data
fred.index = pd.DatetimeIndex(fred.index)
fred[["T5YIE","T10YIE"]] = fred[["T5YIE","T10YIE"]].interpolate(method = "linear")
fred = fred.dropna()
print("Last data point for:" +str(fred.index[-1]))


In [ ]:

monthly_target = 1.02**(1/12)

#5 Years
fred5 = fred[["PCEPI","T5YIE"]].tail(60)
fred5["Target"] = 100

target5_loc = fred5.columns.get_loc("Target")
for i in range(1,len(fred5.index)):
  fred5.iloc[i,target5_loc] = fred5.iloc[i-1,target5_loc]*monthly_target

fred5["PCEPI"] = 100* fred5["PCEPI"]/fred5["PCEPI"][0]
fred5["Shortfall"] = fred5["Target"] - fred5["PCEPI"]

fred["Target"] = 100
#10 Years
target_loc = fred.columns.get_loc("Target")
for i in range(1,len(fred.index)):
  fred.iloc[i,target_loc] = fred.iloc[i-1,target_loc]*monthly_target

fred["PCEPI"] = 100* fred["PCEPI"]/fred["PCEPI"][0]
fred["Shortfall"] = fred["Target"] - fred["PCEPI"]

# New Fed framework & Covid - their impact on Breakevens

Last autumn, the Fed revealed that they would aim to overshoot inflation in the future to make up for the years inflation was below 2%. This is interesting because it means that the target for inflation in the next years will be partially dictated by the inflation we had in the past. Below, I quantify the shortfall of inflation in the last 5 years and 10 years and take compute the inflation that would be necessary to meet an average of 2% inflation looking 5 years and 10 years ahead. This allows a comparison between what the Fed wishes to do and what the market things the Fed will achieve.

Looking at the chained Personal Consumption Expenditure Index ([PCEPI](https://fred.stlouisfed.org/series/PCEPI), see Bullard's explanation [here](https://www.stlouisfed.org/publications/regional-economist/july-2013/cpi-vs-pce-inflation--choosing-a-standard-measure)) we see that inflation feel short both over the short five year and longer ten year horizon against a target of 2% pa.

In [ ]:
fig, axs = plt.subplots(nrows = 2,ncols = 2,sharex="col")
fig.suptitle("Previous 5,10 Years Realized vs Targeted Inflation and Cumulative Shortfall")

axs[0,0].plot(fred5["PCEPI"],label = "PCEPI")
axs[0,0].plot(fred5["Target"],label = "Target")
axs[0,0].grid(True)
axs[0,0].legend(loc="upper left")
axs[1,0].plot(fred5["Shortfall"],label = "Inflation Shortfall")
axs[1,0].grid(True)
axs[1,0].legend(loc="upper left")

axs[0,1].plot(fred["PCEPI"],label = "PCEPI")
axs[0,1].plot(fred["Target"],label = "Target")
axs[0,1].grid(True)
axs[0,1].legend(loc="upper left")
axs[1,1].plot(fred["Shortfall"],label = "Inflation Shortfall")
axs[1,1].grid(True)
axs[1,1].legend(loc="upper left")

#fig.tight_layout()
fig.autofmt_xdate()
plt.show()

In [ ]:
next_month = fred.index[-1]+pd.DateOffset(months=1)

#5 Year
future_index5 = pd.date_range(start = next_month, end = datetime.datetime(year=fred.index[-1].year+5, month=fred.index[-1].month, day=fred.index[-1].day), freq="MS")
future_index = pd.date_range(start = next_month, end = datetime.datetime(year=fred.index[-1].year+10, month=fred.index[-1].month, day=fred.index[-1].day), freq="MS")

target_future5 = fred5["Target"][-1]*monthly_target**60
target_future = fred["Target"][-1]*monthly_target**120

PCEPI_now5 = fred5["PCEPI"][-1]
PCEPI_now = fred["PCEPI"][-1]
necessary_rate5 = (target_future5/PCEPI_now5)**(1/60)
necessary_rate = (target_future/PCEPI_now)**(1/120)

PCEPI_necessary5 = [PCEPI_now5]
target_future_list5 = [fred5["Target"][-1]]
for i in range(1,60):
  PCEPI_necessary5.append(PCEPI_necessary5[i-1]*necessary_rate5)
  target_future_list5.append(target_future_list5[i-1]*monthly_target)

#10 Year
PCEPI_necessary = [PCEPI_now]
target_future_list = [fred["Target"][-1]]
for i in range(1,120):
  PCEPI_necessary.append(PCEPI_necessary[i-1]*necessary_rate)
  target_future_list.append(target_future_list[i-1]*monthly_target)

future_df5 = pd.DataFrame({"PCEPI":PCEPI_necessary5[1:],"Target":target_future_list5[1:]}, index = future_index5)
future_df = pd.DataFrame({"PCEPI":PCEPI_necessary[1:],"Target":target_future_list[1:]}, index = future_index)

combined_df5 = pd.concat([fred5,future_df5])
combined_df5["Shortfall"] = combined_df5["Target"] - combined_df5["PCEPI"]

combined_df = pd.concat([fred,future_df])
combined_df["Shortfall"] = combined_df["Target"] - combined_df["PCEPI"]

If we assume that the Fed will want to reach an average of 2% inflation over either the next 5 years (so 2% over 10 years) or over the next 10 years (so 2% over 20 years), we can calculate the necessary inflation rate to achieve this target. Below, I take the same data as shown above, but add 5, 10 years in which inflation is held constant such that by the end of the period the Fed has achieved an average of 2% inflation.

In [ ]:

fig, axs = plt.subplots(nrows = 2,ncols = 2,sharex="col")
fig.suptitle("Inflation vs Policy Goals 5y, 10y")

axs[0,0].plot(combined_df5["PCEPI"],label = "Realized, necessary PCEPI")
axs[0,0].plot(combined_df5["Target"],label = "Target")
axs[0,0].grid(True)
axs[0,0].legend(loc="lower right", framealpha=0.5)

axs[1,0].plot(combined_df5["Shortfall"],label = "Inflation Shortfall")
axs[1,0].grid(True)
axs[1,0].legend(loc="lower right",framealpha=0.5)

axs[0,1].plot(combined_df["PCEPI"],label = "Realized, necessary PCEPI")
axs[0,1].plot(combined_df["Target"],label = "Target")
axs[0,1].grid(True)
axs[0,1].legend(loc="lower right", framealpha=0.5)

axs[1,1].plot(combined_df["Shortfall"],label = "Inflation Shortfall")
axs[1,1].grid(True)
axs[1,1].legend(loc="lower right",framealpha=0.5)
fig.autofmt_xdate()
#fig.tight_layout()
plt.show()

This raises the question: Does the market believe that the Fed will achieve the goal over the next five or even ten years? Below, I therefore add to the plot the inflation path that markets where pricing when the last PCE data came out `r tail(rownames(py$fred5),1)`.

At the end of February, 5yr break-evens were priced so that it seemed like the market believed the Fed would almost, but not quite achieve its goal over the next 10 years. As of end of March, markets were expecting to overshoot the Feds target over 5 years by almost as much as it was expecting to undershoot it during the height of Covid.

This is *before* we take into account recent data int the week 10th to 16th of May.

However, this is still seen as transitory. In the long run, markets still expect the Fed to fall short as implied by the 10 year breakevens.

In [ ]:

combined_df5["PCEPI_Implied"] = PCEPI_now5

CEPI_implied5_loc = combined_df5.columns.get_loc("PCEPI_Implied")
last_breakeven5 = (1+fred5["T5YIE"][-1]/100)**(1/12)

for i in range(len(fred5.index),len(combined_df5.index)):
  combined_df5.iloc[i,CEPI_implied5_loc] = combined_df5.iloc[i-1,CEPI_implied5_loc]*last_breakeven5

combined_df5["PCEPI_Implied"][0:len(fred5.index)] = None
combined_df5["Implied_Shortfall"] = combined_df5["Target"] - combined_df5["PCEPI_Implied"]

combined_df["PCEPI_Implied"] = PCEPI_now

CEPI_implied_loc = combined_df.columns.get_loc("PCEPI_Implied")
last_breakeven10 = (1+fred["T10YIE"][-1]/100)**(1/12)

for i in range(len(fred.index),len(combined_df.index)):
  combined_df.iloc[i,CEPI_implied_loc] = combined_df.iloc[i-1,CEPI_implied_loc]*last_breakeven10

combined_df["PCEPI_Implied"][0:len(fred.index)] = None
combined_df["Implied_Shortfall"] = combined_df["Target"] - combined_df["PCEPI_Implied"]

In [ ]:
fig, axs = plt.subplots(nrows = 2,ncols = 2,sharex="col")
fig.suptitle("Next 5, 10 Years Necessary vs Targeted Inflation and Cumulative Shortfall")

axs[0,0].plot(combined_df5["PCEPI"],label = "Realized, later necessary PCEPI")
axs[0,0].plot(combined_df5["Target"],label = "Target")
axs[0,0].plot(combined_df5["PCEPI_Implied"],label = "5Y BE Implied")
axs[0,0].grid(True)
axs[0,0].legend(loc="lower left")

axs[1,0].plot(combined_df5["Shortfall"],label = "Shortfall, given target met")
axs[1,0].plot(combined_df5["Implied_Shortfall"],label = "Implied Shortfall")
axs[1,0].grid(True)
axs[1,0].legend(loc="lower left",framealpha=0.5)

axs[0,1].plot(combined_df["PCEPI"],label = "Realized, later necessary PCEPI")
axs[0,1].plot(combined_df["Target"],label = "Target")
axs[0,1].plot(combined_df["PCEPI_Implied"],label = "10Y BE Implied")
axs[0,1].grid(True)
axs[0,1].legend(loc="lower left")

axs[1,1].plot(combined_df["Shortfall"],label = "Shortfall, given target met")
axs[1,1].plot(combined_df["Implied_Shortfall"],label = "Implied Shortfall")
axs[1,1].grid(True)
axs[1,1].legend(loc="lower left",framealpha=0.5)

fig.autofmt_xdate()
#fig.tight_layout()
plt.show()

Comparing the actually expected vs. necessary inflation numbers, we can even see that the ten year inflation expectations are not only lower than they would be if people believed the Fed would be able (or would want to) hit their inflation target, they are even lower than the five year inflation expectations.

In [ ]:

five_year = [fred5["T5YIE"][-1],((necessary_rate5**12)-1)*100]
ten_year = [fred["T10YIE"][-1],((necessary_rate**12)-1)*100]

fig, axs = plt.subplots(nrows = 1,ncols = 2,sharey="row")
fig.suptitle("Next 5, 10 Years Necessary vs Expected Inflation as of: "+str(fred.index[-1]))

axs[0].bar(["5y Breakeven \n"+"currently: "+str(round(five_year[0],2)),"Necessary Inflation:\n "+str(round(five_year[1],2))],five_year)
axs[0].grid(True)
axs[0].title.set_text('5 Year')

axs[1].bar(["10y Breakeven \n"+"currently: "+str(round(ten_year[0],2)),"Necessary Inflation: \n"+str(round(ten_year[1],2))],ten_year)
axs[1].grid(True)
axs[1].title.set_text('10 Year')

#fig.tight_layout()
plt.show()

This raises an interesting question about inflation expectations more generally. Looking at historical breakevens for five years, 10 years and the 5y5y forward implied rate, we can actually see that having lower 5 year expectations than 10 year expectations is quite a novel feature. And this difference has only changed very recently.

In [ ]:
start = datetime.datetime(year=end.year-20, month=end.month, day=end.day)
#symbols = ["T5YIE","T5YIFR","T10YIE"]
#fred_1= pdr.DataReader(symbols, 'fred', start, end)
#fred_1.to_csv("fred_1.csv")
fred = pd.read_csv("fred_1.csv", index_col = "DATE")
fred.index = pd.DatetimeIndex(fred.index)
fred["Target"] = 2
five_five_vs_five = fred["T5YIE"]-fred["T5YIFR"]


In [ ]:
fred.plot(title = "US Breakevens")
plt.show()

Below, I zoom into the time since 2020. Very interestingly the difference between inflation expectations decreased as their overall level increased in H2 2020, but it was only on the 7th of January 2021, 2 days after the Georgia elections, that this relationship turned around for the first time in over 10 years.

In [ ]:

fig, axs = plt.subplots(2,1,sharex="col")
fig.suptitle("Structure of Inflation Expectation changed in January 2021")

axs[0].plot(fred["2020-01-01":])
axs[0].legend(labels = ["5y BE", "5y5y BE","10y BE"])
axs[1].plot(five_five_vs_five["2020-01-01":])
axs[1].legend(["Difference 5y BE - 5y5y BE"])
axs[0].grid(True)
axs[1].grid(True)
fig.autofmt_xdate()
#fig.tight_layout()
plt.show()


As one would expect, the recent increase in medium-term inflation expectations had also had its impact on Fed expectations, with a hike in June being now (actually as of 16.05., so 6 weeks after last data I used in above analysis) priced at 10%.

<https://www.cmegroup.com/trading/interest-rates/countdown-to-fomc.html>

Possible next step:

-\> get more available data by using PPI and CPI to get to PCE estimate